# Введение в MapReduce модель на Python


In [2]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator
import random

In [ ]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [ ]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [ ]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [ ]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [ ]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [ ]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [ ]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [ ]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [ ]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [ ]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [ ]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [ ]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [ ]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [ ]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 2.905589809636405),
 (1, 2.905589809636405),
 (2, 2.905589809636405),
 (3, 2.905589809636405),
 (4, 2.905589809636405)]

## Inverted index

In [ ]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('what', ['0', '1']),
 ('is', ['0', '1', '2']),
 ('it', ['0', '1', '2']),
 ('a', ['2']),
 ('banana', ['2'])]

## WordCount

In [ ]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [4]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [14]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('is', 18), ('what', 10)]),
 (1, [('a', 2), ('banana', 2), ('it', 18)])]

## TeraSort

In [6]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.0686096003285891),
   (None, 0.08264341190771929),
   (None, 0.08561885174475892),
   (None, 0.10246395536698094),
   (None, 0.10687044632898024),
   (None, 0.11369969473542374),
   (None, 0.1168462531945138),
   (None, 0.1535724862549932),
   (None, 0.1598159711471303),
   (None, 0.16299294636632766),
   (None, 0.2120978097247842),
   (None, 0.2392076802464106),
   (None, 0.25589509000525523),
   (None, 0.2630935334369232),
   (None, 0.38688909234296043),
   (None, 0.4063354331851803),
   (None, 0.41348287790359073),
   (None, 0.42213008949931585),
   (None, 0.4486865281856742)]),
 (1,
  [(None, 0.602023832572674),
   (None, 0.6430626698752681),
   (None, 0.6448693695056031),
   (None, 0.6449217963679532),
   (None, 0.6768028099934498),
   (None, 0.7253640431297059),
   (None, 0.9059535695192993),
   (None, 0.9706825402480044),
   (None, 0.9710131351844195),
   (None, 0.9839337152768212),
   (None, 0.991514360505179)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [ ]:
import random

def MAP(num_list):
    return max(num_list)


def REDUCE(num_list):
    return max(num_list)


def RECORDREADER(count):
    return [random.randint(0, 100) for i in range(count)]


record = RECORDREADER(100)
print(record)

part_count = 5
record_partitional = [record[d:d + part_count] for d in range(0, len(record), part_count)]
print(record_partitional)

print(REDUCE(list(map(lambda x: MAP(x), record_partitional))))

[68, 75, 22, 40, 6, 65, 25, 81, 23, 54, 0, 88, 54, 93, 81, 96, 79, 98, 30, 59, 91, 60, 18, 99, 62, 32, 75, 29, 83, 10, 53, 60, 8, 30, 46, 49, 16, 53, 65, 73, 90, 61, 86, 77, 79, 21, 46, 26, 40, 71, 79, 56, 77, 9, 31, 54, 59, 25, 50, 94, 81, 17, 26, 43, 20, 58, 79, 57, 25, 1, 84, 43, 17, 56, 34, 74, 17, 0, 75, 97, 54, 55, 73, 31, 40, 28, 34, 43, 44, 27, 94, 70, 99, 38, 97, 38, 31, 62, 74, 72]
[[68, 75, 22, 40, 6], [65, 25, 81, 23, 54], [0, 88, 54, 93, 81], [96, 79, 98, 30, 59], [91, 60, 18, 99, 62], [32, 75, 29, 83, 10], [53, 60, 8, 30, 46], [49, 16, 53, 65, 73], [90, 61, 86, 77, 79], [21, 46, 26, 40, 71], [79, 56, 77, 9, 31], [54, 59, 25, 50, 94], [81, 17, 26, 43, 20], [58, 79, 57, 25, 1], [84, 43, 17, 56, 34], [74, 17, 0, 75, 97], [54, 55, 73, 31, 40], [28, 34, 43, 44, 27], [94, 70, 99, 38, 97], [38, 31, 62, 74, 72]]
99


### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [ ]:
def MAP(num):
    return (1, num)


def REDUCE(key, nums: Iterator[NamedTuple]):
    sum = 0
    count = 0
    for n in nums:
        sum += n
        count += 1
    yield sum / count


def RECORDREADER(count):
    return [random.randint(0, 100) for i in range(count)]


map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = list(flatten(map(lambda x: REDUCE(*x), shuffle_output)))
print(reduce_output)

[(1, 65), (1, 6), (1, 25), (1, 73), (1, 9), (1, 51), (1, 90), (1, 70), (1, 48), (1, 71), (1, 98), (1, 71), (1, 73), (1, 26), (1, 57), (1, 75), (1, 26), (1, 67), (1, 73), (1, 1), (1, 55), (1, 86), (1, 23), (1, 85), (1, 65), (1, 45), (1, 85), (1, 75), (1, 80), (1, 55), (1, 37), (1, 24), (1, 16), (1, 21), (1, 37), (1, 17), (1, 92), (1, 91), (1, 63), (1, 7), (1, 77), (1, 93), (1, 97), (1, 18), (1, 57), (1, 70), (1, 46), (1, 54), (1, 33), (1, 33), (1, 39), (1, 38), (1, 94), (1, 11), (1, 8), (1, 24), (1, 72), (1, 89), (1, 51), (1, 20), (1, 7), (1, 60), (1, 70), (1, 44), (1, 0), (1, 77), (1, 49), (1, 52), (1, 12), (1, 41), (1, 49), (1, 57), (1, 5), (1, 91), (1, 16), (1, 29), (1, 65), (1, 31), (1, 35), (1, 91), (1, 7), (1, 54), (1, 17), (1, 71), (1, 91), (1, 98), (1, 64), (1, 12), (1, 82), (1, 93), (1, 11), (1, 60), (1, 91), (1, 97), (1, 53), (1, 24), (1, 7), (1, 51), (1, 95), (1, 14)]
[(1, [65, 6, 25, 73, 9, 51, 90, 70, 48, 71, 98, 71, 73, 26, 57, 75, 26, 67, 73, 1, 55, 86, 23, 85, 65, 45, 85

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [ ]:
def group_by_key(iterable):
    sorted_data = sorted(iterable, key=lambda x: x[0])
    grouped_data = {}
    for key, value in sorted_data:
        if key not in grouped_data:
            grouped_data[key] = [value]
        else:
            grouped_data[key].append(value)
    result = []
    for key, value in grouped_data.items():
        result.append((key, value))
    return result

def MAP(num):
    return (1, num)


def REDUCE(key, nums: Iterator[NamedTuple]):
    sum = 0
    count = 0
    for n in nums:
        sum += n
        count += 1
    yield sum / count


def RECORDREADER(count):
    return [random.randint(0, 100) for i in range(count)]

map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = list(flatten(map(lambda x: REDUCE(*x), shuffle_output)))
print(reduce_output)

[(1, 31), (1, 47), (1, 41), (1, 35), (1, 93), (1, 2), (1, 60), (1, 7), (1, 14), (1, 39), (1, 44), (1, 83), (1, 97), (1, 97), (1, 81), (1, 54), (1, 80), (1, 5), (1, 3), (1, 61), (1, 91), (1, 47), (1, 77), (1, 84), (1, 18), (1, 65), (1, 1), (1, 90), (1, 98), (1, 41), (1, 30), (1, 56), (1, 8), (1, 24), (1, 70), (1, 13), (1, 95), (1, 4), (1, 79), (1, 67), (1, 70), (1, 78), (1, 3), (1, 5), (1, 4), (1, 56), (1, 42), (1, 96), (1, 4), (1, 88), (1, 95), (1, 23), (1, 13), (1, 6), (1, 39), (1, 48), (1, 9), (1, 42), (1, 18), (1, 66), (1, 21), (1, 22), (1, 77), (1, 11), (1, 55), (1, 68), (1, 80), (1, 94), (1, 15), (1, 69), (1, 49), (1, 64), (1, 74), (1, 91), (1, 77), (1, 49), (1, 48), (1, 35), (1, 71), (1, 83), (1, 72), (1, 96), (1, 73), (1, 67), (1, 90), (1, 96), (1, 55), (1, 52), (1, 47), (1, 39), (1, 54), (1, 12), (1, 14), (1, 45), (1, 14), (1, 94), (1, 79), (1, 52), (1, 33), (1, 11)]
[(1, [31, 47, 41, 35, 93, 2, 60, 7, 14, 39, 44, 83, 97, 97, 81, 54, 80, 5, 3, 61, 91, 47, 77, 84, 18, 65, 1, 90,

### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [29]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs


# ===================================================================================================================================================================================

from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, word)

def REDUCE(key:str, value:Iterator[str]):
  yield key

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, ['', 'is', 'what']), (1, ['a', 'banana', 'it'])]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [50]:
from collections import defaultdict

def MAP(el_list):
    mapped_result = defaultdict(list)
    for t in el_list:
        if C(t):  # Предикат C
            mapped_result[t].append(t)
    return mapped_result.items()


def REDUCE(mapped_items):
    reduced_result = []
    print(mapped_items)
    for values in mapped_items:
        for value in values:
            reduced_result.append(value[0])
    return reduced_result

def C(t):
    return t[0] % 2 == 0  # Выбираем только кортежи с четным первым элементом


def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for i in range(count)]

record = RECORDREADER(100)
print(record)

part_count = 5
record_partitional = [record[d:d + part_count] for d in range(0, len(record), part_count)]
print(record_partitional)

print(REDUCE(list(map(lambda x: MAP(x), record_partitional))))

[(80, 25, 24), (97, 29, 22), (70, 24, 10), (1, 34, 66), (74, 32, 97), (61, 23, 95), (44, 100, 90), (19, 18, 11), (41, 51, 76), (8, 22, 31), (55, 23, 8), (2, 99, 13), (31, 21, 13), (91, 65, 44), (93, 73, 86), (24, 43, 88), (86, 45, 18), (75, 65, 14), (0, 66, 89), (26, 46, 94), (85, 83, 62), (93, 90, 72), (45, 45, 95), (71, 98, 93), (7, 10, 52), (99, 30, 97), (40, 30, 45), (80, 100, 38), (100, 91, 27), (3, 67, 20), (50, 4, 44), (84, 100, 88), (33, 97, 50), (100, 100, 87), (36, 41, 60), (94, 75, 6), (79, 73, 35), (63, 78, 79), (41, 83, 84), (71, 73, 37), (72, 62, 99), (33, 62, 57), (66, 3, 67), (32, 59, 9), (94, 42, 71), (95, 24, 17), (19, 16, 28), (85, 96, 14), (8, 38, 42), (86, 45, 57), (88, 39, 47), (3, 21, 70), (0, 73, 66), (52, 61, 81), (91, 89, 30), (26, 36, 49), (54, 98, 87), (3, 61, 14), (80, 100, 30), (93, 1, 50), (48, 11, 13), (3, 46, 10), (32, 90, 49), (65, 9, 69), (64, 95, 57), (99, 33, 68), (66, 75, 56), (15, 77, 79), (92, 82, 90), (49, 1, 25), (33, 9, 45), (14, 40, 61), (38,

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [80]:
S = set([1 ,12, 56])

def MAP(t):
  res_list = []
  for el in t:
    if el in S:
      res_list.append(el)
    res = tuple(res_list)
    return (res, res)


def REDUCE(key, values: Iterator[NamedTuple]):
    return (key, key)

def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for i in range(count)]

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x)[0], shuffle_output))
print(reduce_output)

[(18, 88, 100), (61, 24, 65), (8, 56, 80), (18, 27, 35), (54, 90, 74), (60, 42, 72), (76, 25, 34), (4, 17, 84), (71, 7, 16), (99, 33, 70), (35, 31, 95), (29, 46, 39), (92, 24, 87), (26, 58, 13), (88, 40, 41), (49, 58, 45), (59, 7, 12), (27, 33, 24), (64, 85, 86), (56, 96, 47), (52, 74, 22), (81, 96, 82), (23, 46, 20), (85, 90, 51), (27, 82, 67), (15, 35, 20), (15, 87, 53), (39, 5, 41), (55, 4, 22), (73, 95, 61), (15, 70, 99), (90, 49, 70), (75, 90, 83), (50, 95, 98), (87, 3, 31), (59, 71, 34), (23, 19, 6), (75, 53, 100), (28, 47, 35), (7, 41, 89), (56, 30, 63), (94, 84, 37), (8, 17, 98), (99, 41, 1), (56, 85, 41), (92, 86, 21), (33, 88, 2), (31, 60, 70), (39, 63, 33), (20, 57, 9), (60, 26, 23), (44, 66, 20), (99, 99, 60), (67, 42, 76), (24, 49, 33), (30, 100, 98), (100, 77, 82), (35, 70, 49), (7, 27, 13), (55, 13, 10), (39, 8, 64), (75, 7, 64), (40, 67, 27), (46, 64, 65), (65, 57, 60), (90, 17, 30), (14, 98, 14), (46, 73, 54), (0, 85, 50), (83, 70, 85), (85, 77, 45), (21, 80, 44), (91,

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [86]:
def MAP(t):
  return (t, t)


def REDUCE(key, values: Iterator[NamedTuple]):
    return (key, key)

def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for i in range(count)]

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x)[0], shuffle_output))
print(reduce_output)

[(9, 54, 14), (89, 58, 97), (97, 58, 30), (98, 53, 8), (16, 52, 50), (98, 77, 50), (50, 0, 18), (23, 40, 96), (24, 81, 52), (28, 50, 30), (93, 24, 98), (84, 27, 9), (26, 18, 72), (39, 77, 11), (87, 29, 67), (25, 87, 31), (85, 42, 28), (78, 27, 23), (76, 51, 76), (55, 51, 72), (60, 91, 24), (96, 24, 27), (16, 1, 12), (5, 38, 33), (10, 1, 11), (59, 37, 45), (82, 45, 65), (73, 6, 47), (59, 79, 74), (25, 73, 93), (3, 11, 36), (61, 51, 48), (7, 53, 54), (77, 98, 24), (100, 19, 50), (28, 24, 41), (60, 52, 39), (52, 79, 13), (42, 86, 44), (97, 84, 52), (74, 39, 85), (94, 12, 71), (80, 21, 43), (17, 77, 30), (58, 38, 77), (54, 80, 59), (46, 82, 6), (21, 37, 20), (95, 24, 24), (61, 46, 5), (82, 83, 20), (83, 23, 94), (77, 95, 86), (20, 71, 17), (31, 43, 37), (8, 18, 5), (45, 78, 1), (23, 38, 36), (27, 10, 10), (8, 70, 97), (12, 27, 76), (54, 1, 79), (67, 34, 45), (40, 67, 83), (40, 82, 26), (8, 71, 8), (92, 28, 98), (1, 77, 21), (23, 70, 94), (67, 9, 41), (48, 36, 8), (93, 35, 57), (18, 26, 59)

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [120]:
def MAP(t):
  return (t, t)


def REDUCE(key, values: Iterator[NamedTuple]):
  if len(values) == 2:
    return (key, key)

def RECORDREADER(count):
    return [(random.randint(0, 3), random.randint(0, 3)) for i in range(count)]

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = [el[0] for el in list(map(lambda x: REDUCE(*x), shuffle_output)) if el is not None]
print(reduce_output)

[(0, 2), (0, 3), (1, 2), (2, 1), (2, 0), (1, 2), (1, 3), (3, 2), (2, 1), (0, 1), (0, 2), (3, 2), (0, 1), (1, 3), (2, 1), (2, 2), (1, 1), (2, 0), (2, 3), (2, 1), (2, 1), (1, 2), (1, 3), (1, 0), (1, 1), (3, 0), (0, 2), (3, 0), (3, 0), (1, 0), (2, 0), (3, 2), (2, 3), (3, 1), (3, 3), (3, 2), (3, 0), (2, 0), (3, 2), (2, 3), (3, 1), (1, 2), (1, 3), (2, 0), (2, 1), (0, 1), (0, 3), (3, 1), (1, 0), (3, 2), (1, 3), (3, 3), (2, 0), (0, 0), (3, 2), (3, 2), (2, 0), (3, 2), (2, 0), (3, 0), (3, 2), (3, 3), (3, 2), (3, 1), (2, 1), (0, 2), (0, 2), (3, 1), (0, 1), (2, 2), (3, 1), (2, 0), (3, 3), (2, 2), (1, 3), (3, 3), (3, 2), (3, 0), (2, 0), (0, 2), (3, 2), (0, 0), (1, 1), (0, 3), (3, 3), (1, 2), (0, 2), (2, 2), (3, 1), (2, 1), (0, 0), (0, 1), (0, 3), (0, 3), (0, 0), (1, 1), (0, 3), (3, 0), (2, 2), (0, 1)]
[((2, 2), (2, 2)), ((1, 1), (1, 1)), ((0, 0), (0, 0)), ((3, 3), (3, 3)), ((1, 0), (1, 0)), ((1, 0), (1, 0)), ((3, 3), (3, 3)), ((0, 3), (0, 3)), ((3, 2), (3, 2)), ((1, 3), (1, 3)), ((3, 3), (3, 3)), 

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [128]:
rels = [1, 2]
class Tuple: # класс кортежа, который хранит данные и имя представления из массива rels выше
  def __init__(self, data: tuple, rel_id: int):
    self.data = data
    self.rel_id = rel_id

def get_random_tuple(count):
  data = tuple([(random.randint(0, 3), random.randint(0, 3)) for i in range(count)])
  rel_id = rels[random.randint(0, len(rels) - 1)]
  return Tuple(data, rel_id)


def RECORDREADER(count):
    return [get_random_tuple(3) for i in range(count)]

def MAP(t: Tuple):
  return (t.data, t.rel_id)

def REDUCE(key, values: Iterator[NamedTuple]):
  if values == [rels[0]]: # если соответствующее значение является списком  [R] , создайте пару  (t,t)
    return (key, key)

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = [el[0] for el in list(map(lambda x: REDUCE(*x), shuffle_output)) if el is not None]
print(reduce_output)

[<__main__.Tuple object at 0x7f3aea780fa0>, <__main__.Tuple object at 0x7f3aea781030>, <__main__.Tuple object at 0x7f3aea782dd0>, <__main__.Tuple object at 0x7f3aea781060>, <__main__.Tuple object at 0x7f3aea783e50>, <__main__.Tuple object at 0x7f3aea782fb0>, <__main__.Tuple object at 0x7f3aea782440>, <__main__.Tuple object at 0x7f3aea781540>, <__main__.Tuple object at 0x7f3aea782530>, <__main__.Tuple object at 0x7f3aea780ee0>, <__main__.Tuple object at 0x7f3aea7807c0>, <__main__.Tuple object at 0x7f3aea781840>, <__main__.Tuple object at 0x7f3aea898b20>, <__main__.Tuple object at 0x7f3aea89b6a0>, <__main__.Tuple object at 0x7f3aea89a4a0>, <__main__.Tuple object at 0x7f3aea898d60>, <__main__.Tuple object at 0x7f3aea899a80>, <__main__.Tuple object at 0x7f3aea89a9b0>, <__main__.Tuple object at 0x7f3aea8996c0>, <__main__.Tuple object at 0x7f3aea89ac20>, <__main__.Tuple object at 0x7f3aea89a560>, <__main__.Tuple object at 0x7f3aea8987c0>, <__main__.Tuple object at 0x7f3aea8993f0>, <__main__.

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [17]:
rels = [1, 2]
class Tuple: # класс кортежа, который хранит данные и имя представления из массива rels выше
  def __init__(self, data: tuple, rel_id: int):
    self.data = data
    self.rel_id = rel_id

def get_random_tuple():
  data = (random.randint(0, 3), random.randint(0, 3))
  rel_id = rels[random.randint(0, len(rels) - 1)]
  return Tuple(data, rel_id)


def RECORDREADER(count):
  return [get_random_tuple() for i in range(count)]

def MAP(t: Tuple):
  if t.rel_id == rels[0]:
    return (t.data[1], (t.rel_id, t.data[0]))
  else:
    return (t.data[0], (t.rel_id, t.data[1]))

def REDUCE(key, values: Iterator[NamedTuple]):
  res = []
  for v in values:
    res.append((v[0], key, v[1]))
  return res

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x), shuffle_output))
print(reduce_output)

[<__main__.Tuple object at 0x7cdd86206cb0>, <__main__.Tuple object at 0x7cdd86207310>, <__main__.Tuple object at 0x7cdd86205540>, <__main__.Tuple object at 0x7cdd86206ce0>, <__main__.Tuple object at 0x7cdd86204af0>, <__main__.Tuple object at 0x7cdd86205240>, <__main__.Tuple object at 0x7cdd86207f10>, <__main__.Tuple object at 0x7cdd86205bd0>, <__main__.Tuple object at 0x7cdd86205db0>, <__main__.Tuple object at 0x7cdd861ad4e0>, <__main__.Tuple object at 0x7cdd861adb70>, <__main__.Tuple object at 0x7cdd861ae800>, <__main__.Tuple object at 0x7cdd861afdc0>, <__main__.Tuple object at 0x7cdd861aebf0>, <__main__.Tuple object at 0x7cdd861af8b0>, <__main__.Tuple object at 0x7cdd861af340>, <__main__.Tuple object at 0x7cdd861ad390>, <__main__.Tuple object at 0x7cdd861acfd0>, <__main__.Tuple object at 0x7cdd861acd60>, <__main__.Tuple object at 0x7cdd861aff70>, <__main__.Tuple object at 0x7cdd861adf30>, <__main__.Tuple object at 0x7cdd861ac700>, <__main__.Tuple object at 0x7cdd861ad8d0>, <__main__.

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [19]:
def get_random_tuple():
  return (random.randint(0, 3), random.randint(0, 3), random.randint(0, 3))

def RECORDREADER(count):
  return [get_random_tuple() for i in range(count)]

def MAP(t: Tuple):
  return (t[0], t[1])

def tetta(values):
  return sum(values)


def REDUCE(key, values: Iterator[NamedTuple]):
  x = tetta(values)
  return (key, x)

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x), shuffle_output))
print(reduce_output)

[(1, 3, 3), (0, 3, 0), (2, 2, 2), (3, 2, 0), (0, 3, 1), (3, 3, 2), (1, 1, 0), (2, 2, 2), (0, 3, 3), (2, 1, 0), (2, 2, 2), (0, 1, 0), (0, 0, 1), (3, 2, 1), (2, 0, 0), (3, 2, 0), (3, 1, 3), (1, 1, 1), (0, 2, 3), (2, 3, 2), (3, 2, 1), (2, 3, 2), (2, 3, 3), (0, 3, 2), (1, 1, 3), (2, 2, 1), (2, 1, 1), (2, 0, 3), (2, 0, 1), (1, 0, 2), (3, 3, 0), (1, 1, 0), (1, 3, 3), (0, 3, 1), (1, 1, 3), (1, 1, 0), (3, 3, 0), (3, 1, 0), (2, 3, 0), (0, 3, 0), (2, 2, 2), (3, 3, 3), (2, 1, 2), (0, 2, 0), (3, 2, 3), (3, 3, 3), (0, 0, 2), (0, 2, 0), (2, 1, 2), (2, 0, 1), (0, 1, 2), (1, 1, 1), (1, 0, 2), (1, 2, 0), (0, 2, 0), (2, 2, 3), (2, 0, 2), (0, 3, 0), (1, 3, 2), (0, 2, 2), (1, 0, 3), (0, 0, 1), (1, 2, 2), (0, 2, 0), (0, 2, 3), (0, 2, 0), (2, 3, 3), (1, 1, 3), (2, 2, 2), (3, 3, 2), (0, 3, 2), (3, 3, 1), (0, 2, 3), (1, 1, 3), (1, 0, 2), (1, 3, 3), (1, 2, 3), (0, 0, 3), (0, 0, 1), (1, 3, 1), (2, 2, 2), (3, 1, 3), (1, 2, 1), (2, 3, 1), (3, 1, 1), (3, 1, 1), (0, 1, 1), (1, 1, 2), (1, 0, 0), (1, 3, 0), (2, 2, 3)

#

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [57]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [18]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j, k), big_mat[j,k])

def MAP(k1, v1):
  (j, k) = k1
  w = v1
  for i in range(small_mat.shape[0]):
    yield ((i, k), w * small_mat[i][j])

def REDUCE(key, values):
  (i, k) = key
  el_value = 0
  for v in values:
    el_value += v
  yield ((i, k), el_value)

Проверьте своё решение

In [19]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [21]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [91]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)
reference_solution = np.matmul(small_mat, big_mat)

def RECORDREADER():
  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      yield ((0, i, j), small_mat[i,j]) # первая матрица

  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((1, j, k), big_mat[j,k]) # вторая матрица

def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1
  if mat_num == 0:
    yield (j, (mat_num, i, w))
  else:
    yield (i, (mat_num, j, w))


def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]
  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])


def MAP_MUL(k1, v1):
  (i, k) = k1
  yield ((i, k), v1)


def REDUCE_MUL(key, values):
  res_el_value = 0
  for v in values:
    res_el_value += v
  yield (key, res_el_value)


def GET_JOINED():
  for j in joined:
    yield j

joined = MapReduce(RECORDREADER, MAP_JOIN, REDUCE_JOIN)
solution = MapReduce(GET_JOINED, MAP_MUL, REDUCE_MUL)
np.allclose(reference_solution, asmatrix(solution)) # should return true

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [49]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

# ===================================================================================================================================================================================

import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)
reference_solution = np.matmul(small_mat, big_mat)

def INPUTFORMAT():
  first_mat = []
  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      first_mat.append(((0, i, j), small_mat[i,j])) # первая матрица
  yield first_mat

  second_mat = []
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      second_mat.append(((1, j, k), big_mat[j,k])) # вторая матрица
  yield second_mat


def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1
  if mat_num == 0:
    yield (j, (mat_num, i, w))
  else:
    yield (i, (mat_num, j, w))


def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]
  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])


def GET_JOINED():
  for j in joined:
    print("aa", j)
    yield j[1]


def MAP_MUL(k1, v1):
  yield (k1, v1)


def REDUCE_MUL(key, values):
  res_val = 0
  for v in values:
    res_val += v
  yield (key, res_val)

maps = 4
reducers = 2
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)

solution = []
for p in pre_result:
  for v in p[1]:
    solution.append(v)

print(solution)
np.allclose(reference_solution, asmatrix(solution)) # should return true


126 key-value pairs were sent over a network.
[(0, [((0, 0), 0.36711979631288144), ((0, 1), 0.26279825509084176), ((0, 2), 0.129960295119967), ((0, 3), 0.171368937936973), ((0, 4), 0.04377328496095276), ((0, 5), 0.35393248887428624), ((0, 6), 0.05586683073485941), ((0, 7), 0.3464199263574519), ((0, 8), 0.05226559099229944), ((0, 9), 0.0641502449623753), ((0, 10), 0.05787737054815592), ((0, 11), 0.3295986243069457), ((0, 12), 0.23627962081034626), ((0, 13), 0.011021549448558722), ((0, 14), 0.19484620975663614), ((0, 15), 0.33123446643624144), ((0, 16), 0.3503622566448434), ((0, 17), 0.07181455926767266), ((0, 18), 0.28815269412472466), ((0, 19), 0.0699968810900198), ((0, 20), 0.3106463685424101), ((0, 21), 0.1485421549065724), ((0, 22), 0.37705715417082625), ((0, 23), 0.18871665390890344), ((0, 24), 0.08847454180194629), ((0, 25), 0.12876791507798496), ((0, 26), 0.3874878488119588), ((0, 27), 0.13778241807508956), ((0, 28), 0.11828919446904443), ((0, 29), 0.2724645668499338), ((0, 30), 

True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [48]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

# ===================================================================================================================================================================================

import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)
reference_solution = np.matmul(small_mat, big_mat)

def INPUTFORMAT():
  first_mat = []
  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      first_mat.append(((0, i, j), small_mat[i,j])) # первая матрица

  global maps
  split_size =  int(np.ceil(len(first_mat)/maps))
  for i in range(0, len(first_mat), split_size):
    yield first_mat[i:i+split_size]

  second_mat = []
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      second_mat.append(((1, j, k), big_mat[j,k])) # вторая матрица

  split_size =  int(np.ceil(len(second_mat)/maps))
  for i in range(0, len(second_mat), split_size):
    yield second_mat[i:i+split_size]


def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1
  if mat_num == 0:
    yield (j, (mat_num, i, w))
  else:
    yield (i, (mat_num, j, w))


def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]
  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])


def GET_JOINED():
  for j in joined:
    print("aa", j)
    yield j[1]


def MAP_MUL(k1, v1):
  yield (k1, v1)


def REDUCE_MUL(key, values):
  res_val = 0
  for v in values:
    res_val += v
  yield (key, res_val)

maps = 3
reducers = 2
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)

solution = []
for p in pre_result:
  for v in p[1]:
    solution.append(v)

print(solution)
np.allclose(reference_solution, asmatrix(solution)) # should return true


126 key-value pairs were sent over a network.
[(0, [((0, 0), 0.7751722819524031), ((0, 1), 0.7189247031122328), ((0, 2), 0.5280051068692923), ((0, 3), 0.458462676543293), ((0, 4), 0.031779311404463044), ((0, 5), 0.6978090882832404), ((0, 6), 0.6695177690151282), ((0, 7), 0.2775095795634109), ((0, 8), 0.5903236551219502), ((0, 9), 0.7768236340819892), ((0, 10), 0.6549047077156971), ((0, 11), 0.09380195452568817), ((0, 12), 0.7491940048665042), ((0, 13), 0.3057306770202348), ((0, 14), 0.6607584571525843), ((0, 15), 0.24607366556505406), ((0, 16), 0.6402283179164573), ((0, 17), 0.7236720119932946), ((0, 18), 0.04912697122506983), ((0, 19), 0.8497228462900142), ((0, 20), 0.10730740393415564), ((0, 21), 0.1306125833752874), ((0, 22), 0.06023100197138961), ((0, 23), 0.36025351858571947), ((0, 24), 0.4930477165827084), ((0, 25), 0.13993162344660626), ((0, 26), 0.902172266212365), ((0, 27), 0.9493451712317909), ((0, 28), 0.23649990981646268), ((0, 29), 0.7452088441898338), ((0, 30), 0.91779470

True